In [1]:
# Libro para creacion de las dimensiones
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client-2.5.3.jar pyspark-shell'

In [2]:
#importar librerias
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import functions as f
import mysql.connector
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

In [3]:
spark_context = SparkContext()
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

In [4]:
PATH = "./data/"

In [5]:
#---------------------------- EXTRAER ------------------------------------------

vuelos_e2f = spark.read.load(PATH+"vuelos_e2f.csv",format="csv", sep=",", inferSchema="true", header="true")
aeropuertos_c_infra = spark.read.load(PATH+"aeropuertos_c_infra.csv",format="csv", sep=",", inferSchema="true", header="true")
cobertura_aerea = spark.read.load(PATH+"cobertura_aerea.csv",format="csv", sep=",", inferSchema="true", header="true")
matriz_distancia = spark.read.load(PATH+"matriz_distancia.csv",format="csv", sep=",", inferSchema="true", header="true")
RegistroDeVuelos = spark.read.load(PATH+"RegistroDeVuelos.csv",format="csv", sep=",", inferSchema="true", header="true")

In [7]:
# Configuración servidor base de datos multidimensional
user_md = 'root'
psswd_md = ''
db_multidimensional_connection_string = 'jdbc:mysql://localhost:3306/WWImportersDWH'

In [8]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import functions as f
import mysql.connector

def start_spark():
    """Inicializa una sesión de spark. 
    """
    spark_context = SparkContext()
    sql_context = SQLContext(spark_context)
    return sql_context.sparkSession


In [9]:
#-------------------------Transformar la informacion------------------------------------------------#

vuelo=vuelos_e2f.select('id_vuelos','origen', 'destino', 'tipo_equipo','tipo_vuelo','trafico', 'empresa')
vuelo=vuelos_e2f.selectExpr('id_vuelos', 'origen','destino','tipo_equipo as tipo_avion', 'tipo_vuelo', 'empresa')
aeropuerto=aeropuertos_c_infra.selectExpr('id_aeropuerto', 'sigla as Aerodromo', 'nombre as nombre_aeropuerto', 'categoria', 'clase', 'tipo', 'municipio','departamento','latitud', 'longitud', 'fecha_construccion', 'fecha_vigencia')
fecha=vuelos_e2f.select('id_fecha', 'mes','ano', 'fecha')

cobertura=cobertura_aerea.selectExpr('Codigo_Centro_Poblado as id_cobertura', 'Nombre_centro_poblado as municipio','Aeropuerto as aeropuerto', 'Cobertura as cobertura',  'Tipo_Cobertura as tipo_cobertura','distancia', 'D_Regional','D_Aerodromo', 'D_Nacional', 'D_Internacional')
ubicacion_geografica=cobertura_aerea.join(aeropuertos_c_infra.withColumnRenamed('sigla','Aerodromo'), how='left', on='Aerodromo')


In [21]:
# Cambiar formato para editar tabla
aeropuerto=aeropuerto.toPandas()
fecha=fecha.toPandas()

In [22]:
# Agregar fecha
fecha['fecha_vigencia']=aeropuerto['fecha_vigencia']

In [24]:
# select para la creacion de tabla de hechos
registro_vuelos=RegistroDeVuelos.select('id_vuelos','id_aeropuerto','id_fecha','id_cobertura','id_ubicacion','vuelos', 'sillas', 'pasajeros', 'carga_bordo','carga_ofrecida','porc_carga_bordo','porcentaje_vuelos', 'porc_ocupacion_pasajeros','longitud_pista', 'ancho_pista','numero_vuelos_origen')

In [26]:
# Cambio de formato para continuar con la edicion
RegistroDeVuelos.toPandas()

,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,porcentaje_vuelos,porc_carga_bordo,porc_ocupacion_pasajeros,id_vuelos,id_fecha,...,ancho_pista,numero_vuelos_origen,id_aeropuerto,id_ubicacion,D_Regional,distancia,D_Aerodromo,D_Nacional,D_Internacional,id_cobertura
0,1,0.0,0.0,7.0,0.0,0.000084,0.000000,0.000000,1,1,...,19.0,2.0,1,1,62.450943,3.079673,52.671795,3.079673,19.783628,5001000
1,1,0.0,0.0,8.0,0.0,0.000084,0.000000,0.000000,2,2,...,38.0,2.0,2,2,49.859215,17.554548,61.183426,17.554548,35.590645,5001001
2,1,0.0,0.0,8.0,0.0,0.000084,0.000000,0.000000,3,3,...,17.0,24.0,3,3,71.827120,9.853178,49.776710,9.853178,10.069007,5001004
3,1,0.0,0.0,6.0,0.0,0.000084,0.000000,0.000000,4,4,...,5.0,24.0,4,4,57.800800,4.228790,50.078598,4.228790,23.640644,5001009
4,1,0.0,0.0,8.0,0.0,0.000084,0.000000,0.000000,5,5,...,5.0,12.0,5,5,56.869186,5.142963,51.181305,5.142964,24.658146,5001010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,2,194.0,11598.0,82.0,1298.0,0.000168,11.191585,42.268041,859,859,...,8.0,27.0,859,859,71.189660,14.464342,14.464342,147.449577,38.211010,13001003
859,3,291.0,12064.0,178.0,1236.0,0.000252,10.245358,61.168385,860,860,...,8.0,46.0,860,860,115.679344,5.963473,5.963473,195.765896,15.988497,13001004
860,11,1067.0,46090.0,637.0,1189.0,0.000926,2.579735,59.700094,861,861,...,28.0,39.0,861,861,90.307915,9.972866,9.972866,168.667338,15.438510,13001005
861,2,194.0,17430.0,21.0,181.0,0.000168,1.038439,10.824742,862,862,...,48.0,54.0,862,862,92.742035,11.582367,13.188554,171.677643,11.582367,13001006


In [27]:
# Limpieza de columnas innecesarias
RegistroDeVuelos=RegistroDeVuelos.drop('_c0')

In [44]:
#Exportar dimensiones para procesarlas en power Bi
vuelo.toPandas().to_csv(PATH+'dimension_vuelos.csv')
ubicacion_geografica.toPandas().to_csv(PATH+'ubicacion_geografica.csv')
aeropuerto.to_csv(PATH+'dimension_aeropuerto.csv')
fecha.to_csv(PATH+'dimension_fecha.csv')
cobertura.toPandas().to_csv(PATH+'dimension_cobertura.csv')
